<img src='letscodebr_cover.jpeg' align='left' width=100%/>

# Ada Tech [DS-PY-004] Técnicas de Programação I (PY) Aulas 4 e 5 : Pandas - Pivot Table.

## Intro

Uma tabela pivot ou tabela dinâmica é uma ferramenta de resumo que geralmente está disponível em programas de planilha.

Cria medidas de resumo por uma ou mais chaves, usando essas chaves como rótulos de linha ou coluna.

O Pandas fornece dois métodos para a `pivot table` DataFrames.

- [`pandas.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.pivot_table.html) 

- [`pandas.DataFrame.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html)

## Dataset

Nesta prática guiada, usaremos os dados da competição "titanic".

Este conjunto de dados também pode ser baixado em [mwaskom/seaborn-data/titanic.csv](https://github.com/mwaskom/seaborn-data/blob/master/titanic.csv).

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
data = sns.load_dataset('titanic')

Vejamos o tamanho e os primeiros registros do conjunto de dados:

In [2]:
data.shape

(891, 15)

In [3]:
data.head(3)

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True


## `pivot_table`

Analisamos a sobrevivência por sexo e classe.

O valor padrão (default) do argumento aggfunc é a média para o método: [`pandas.DataFrame.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html)

In [4]:
data.pivot_table('survived', index = 'sex', columns = 'class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


Os argumentos para o método pivot_table são:

* o campo no qual vamos calcular a função de agregação, neste caso `média`
* index define a coluna do DataFrame original cujos valores servem como o índice da tabela dinâmica
* columns define a coluna do DataFrame original cujos valores servem como colunas da tabela dinâmica

Nesta tabela, vemos que mais mulheres do que homens sobreviveram, e quanto maior a classe alta, maior proporção de sobreviventes.

## `pivot_table` multi level

Vamos analisar os grupos de idade como uma terceira dimensão.
 
Para isso, geramos categorias dependendo dos valores da variável 'age' usando `cut`

A função [`pandas.cut()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.cut.html) divide os valores de uma variável em intervalos discretos, com os limites especificados no argumento` bins`.

In [5]:
age_categories = pd.cut(data.age, [0, 18, 80])
data.pivot_table('survived', ['sex', age_categories], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

Vamos adicionar informações sobre as categorias que definem a taxa da variável `fare` de acordo com o quantil ao qual cada valor pertence.

Usamos a função [`pandas.qcut()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.qcut.html) para calcular essas categorias.

Definimos duas categorias de `fare`.

In [6]:
fare_categories = pd.qcut(data.fare, 2)
fare_categories

0       (-0.001, 14.454]
1      (14.454, 512.329]
2       (-0.001, 14.454]
3      (14.454, 512.329]
4       (-0.001, 14.454]
             ...        
886     (-0.001, 14.454]
887    (14.454, 512.329]
888    (14.454, 512.329]
889    (14.454, 512.329]
890     (-0.001, 14.454]
Name: fare, Length: 891, dtype: category
Categories (2, interval[float64, right]): [(-0.001, 14.454] < (14.454, 512.329]]

In [7]:
data.pivot_table('survived', ['sex', age_categories], [fare_categories, 'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

Podemos especificar diferentes funções de agregação para diferentes colunas do DataFrame original:

In [8]:
data.pivot_table(index = 'sex', 
                 columns = 'class',
                 aggfunc = {'survived':sum, 'fare':'mean'}
                )

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

Para calcular os totais por grupo, usamos `margins` e` margins_name`:

In [9]:
data.pivot_table('survived', 
                 index = 'sex', 
                 columns = 'class', 
                 margins = True, 
                 margins_name = "Total"
                )

class,First,Second,Third,Total
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
Total,0.629630,0.472826,0.242363,0.383838


#### Referências

Python for Data Analysis. Wes McKinney. Cap 10

[`pandas.DataFrame.pivot_table()`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.pivot_table.html)